In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [29]:
file_path = 'connect4_dataset.csv'
dataset = pd.read_csv(file_path)

dataset.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.32,0.33,0.34,-1,-1.1,0.35,0.36,1.1,1.2,0.37
0,0,0,0,0,0,0,0,1,0,0,...,1.0,0.0,1.0,1.0,-1.0,-1.0,0.0,-1.0,1.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,1.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,-1.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,0.0


In [59]:
# Preprocess data and train model
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

valid_indices = (y >= 0) & (y <= 6)
X = X[valid_indices]
y = y[valid_indices]

y = to_categorical(y, num_classes=7)

X = X.reshape(-1, 6, 7, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    # First convolutional block
    Conv2D(32, (3, 3), activation='tanh', padding='same', input_shape=(6, 7, 1)),
    BatchNormalization(),  # Normalize activations for stable training
    MaxPooling2D((2, 2), padding='same'),

    # Second convolutional block
    Conv2D(64, (3, 3), activation='tanh', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),

    # Flatten and Dense layers
    Flatten(),
    Dense(64, activation='tanh'),  # Reduced dense layer size
    Dropout(0.5),  # Regularization
    Dense(7, activation='softmax')  # Output layer for 7 possible moves
])

optimizer = Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=256)


Epoch 1/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.6326 - loss: 1.3935 - val_accuracy: 0.7030 - val_loss: 1.1663
Epoch 2/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.6956 - loss: 0.9806 - val_accuracy: 0.7030 - val_loss: 1.3340
Epoch 3/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.7147 - loss: 0.8703 - val_accuracy: 0.7028 - val_loss: 1.2915
Epoch 4/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.7478 - loss: 0.7735 - val_accuracy: 0.7010 - val_loss: 1.4788
Epoch 5/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.7752 - loss: 0.7052 - val_accuracy: 0.7013 - val_loss: 1.0759
Epoch 6/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.7997 - loss: 0.6452 - val_accuracy: 0.7950 - val_loss: 0.6569
Epoch 7/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.8123 - loss: 0.6080 - val_accuracy: 0.8148 - val_loss: 0.6131
Epoch 8/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.8248 - loss: 0.5722 - 

In [61]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7982 - loss: 0.9095
Test Accuracy: 80.75%


In [62]:
# Save the trained model to a file
model.save("connect4_cnn_model.h5")

In [63]:
#Play against the bot
def print_board(board):
    """Display the Connect-4 board."""
    symbols = {1: "X", -1: "O", 0: "."}
    for row in board:
        print(" ".join(symbols[cell] for cell in row))
    print("\n")

def is_valid_move(board, col):
    """Check if a move is valid."""
    return board[0, col] == 0

def make_move(board, col, player):
    """Place a player's piece in the chosen column."""
    for row in range(5, -1, -1):
        if board[row, col] == 0:
            board[row, col] = player
            return

def get_bot_move(model, board):
    """Get the bot's move using the trained model, ensuring it's valid."""
    input_board = board.reshape(1, 6, 7, 1)
    predictions = model.predict(input_board)[0]
    valid_columns = [col for col in range(7) if is_valid_move(board, col)]

    masked_predictions = np.array([-1 if col not in valid_columns else predictions[col] for col in range(7)])

    return np.argmax(masked_predictions)

def is_winner(board, player):
    """Check if a player has won."""
    for row in range(6):
        for col in range(7):
            if (
                col + 3 < 7 and all(board[row, col + i] == player for i in range(4))
                or row + 3 < 6 and all(board[row + i, col] == player for i in range(4))
                or row + 3 < 6 and col + 3 < 7 and all(board[row + i, col + i] == player for i in range(4))
                or row + 3 < 6 and col - 3 >= 0 and all(board[row + i, col - i] == player for i in range(4))
            ):
                return True
    return False

def play_game():
    """Play a game of Connect-4 against the bot."""
    model = load_model("connect4_cnn_model.h5")
    board = np.zeros((6, 7), dtype=int)
    print("Welcome to Connect-4! You are 'X' (1). Bot is 'O' (-1).\n")
    print_board(board)
    while True:
        while True:
            try:
                col = int(input("Enter your move (0-6): "))
                if 0 <= col <= 6 and is_valid_move(board, col):
                    break
                else:
                    print("Invalid move. Try again.")
            except ValueError:
                print("Please enter a number between 0 and 6.")
        make_move(board, col, 1)
        print("Your move:")
        print_board(board)
        if is_winner(board, 1):
            print("Congratulations! You win!")
            break

        if not any(board[0, :] == 0):
            print("It's a draw!")
            break

        bot_col = get_bot_move(model, board)
        make_move(board, bot_col, -1)
        print(f"Bot chooses column {bot_col}:")
        print_board(board)
        if is_winner(board, -1):
            print("Bot wins! Better luck next time!")
            break

play_game()


Welcome to Connect-4! You are 'X' (1). Bot is 'O' (-1).

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .


Enter your move (0-6): 3
Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Bot chooses column 0:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . X . . .


Enter your move (0-6): 3
Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
O . . X . . .


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Bot chooses column 0:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . X . . .
O . . X . . .


Enter your move (0-6): 3
Your move:
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
O . . X . . .
O . . X . . .


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Bot chooses column 3:
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . .
O . . X . . .
O . . X . . .


Enter your move (0-6): 0
Your move:
. . . . . . .
. 